# Jeu de données entreprise

Voici le script pour récupérer les colonnes spécifiques siret, categorieentreprise, sectionetablissement, et geolocetablissement, tout en sélectionnant uniquement les enregistrements avec l'état administratif "Actif" dans la colonne etatadministratifetablissement :

On cherche ici à voir la structure du code JSON : 

In [20]:
import requests
import json

base_url = "https://data.paysdelaloire.fr/api/explore/v2.1/catalog/datasets/120027016_base-sirene-v3-ss/records"
limit = 20  # Limiting to 20 for inspection
offset = 0  # Initial offset

# Build the URL with the offset parameter
url = f"{base_url}?limit={limit}&offset={offset}"
print(f"Appel de l'API avec l'URL: {url}")

# Call the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=4))  # Pretty-print the JSON response
else:
    print(f"Erreur lors de l'appel de l'API: {response.status_code}")


Appel de l'API avec l'URL: https://data.paysdelaloire.fr/api/explore/v2.1/catalog/datasets/120027016_base-sirene-v3-ss/records?limit=20&offset=0
{
    "total_count": 1769825,
    "results": [
        {
            "siren": "753569839",
            "nic": "00024",
            "siret": "75356983900024",
            "statutdiffusionetablissement": "O",
            "datecreationetablissement": "2016-01-01",
            "trancheeffectifsetablissement": null,
            "trancheeffectifsetablissementtriable": null,
            "anneeeffectifsetablissement": null,
            "activiteprincipaleregistremetiersetablissement": null,
            "datederniertraitementetablissement": "2024-03-30T10:55:44+00:00",
            "etablissementsiege": "non",
            "nombreperiodesetablissement": 2,
            "complementadresseetablissement": null,
            "numerovoieetablissement": null,
            "indicerepetitionetablissement": null,
            "typevoieetablissement": null,
          

In [21]:
import requests
import pandas as pd

base_url = "https://data.paysdelaloire.fr/api/explore/v2.1/catalog/datasets/120027016_base-sirene-v3-ss/records"
limit = 100  # Nombre de résultats par page
offset = 0  # Décalage initial
df_entreprise = pd.DataFrame()  # DataFrame pour stocker tous les enregistrements

while True:
    # Construire l'URL avec le paramètre de décalage
    url = f"{base_url}?limit={limit}&offset={offset}"
    print(f"Appel de l'API avec l'URL: {url}")

    # Appeler l'API
    response = requests.get(url)

    # Vérifier si la requête a réussi
    if response.status_code != 200:
        print(f"Erreur lors de l'appel de l'API: {response.status_code}")
        break

    data = response.json()

    # Obtenir les résultats
    results = data.get('results', [])
    if results:
        # Normaliser les résultats JSON en DataFrame
        df_results = pd.json_normalize(results)
        
        # Sélectionner les colonnes nécessaires
        try:
            df_results = df_results[['siret', 'categorieentreprise', 'sectionetablissement', 'geolocetablissement.lon', 'geolocetablissement.lat', 'etatadministratifetablissement']]
            df_results = df_results[df_results['etatadministratifetablissement'] == 'Actif']
            
            # Renommer les colonnes pour une meilleure lisibilité
            df_results.columns = ['siret', 'categorieentreprise', 'sectionetablissement', 'longitude', 'latitude', 'etatadministratifetablissement']
            
            df_entreprise = pd.concat([df_entreprise, df_results], ignore_index=True)
        except KeyError as e:
            print(f"KeyError: {e}")
            print("Colonnes disponibles:", df_results.columns)

    # Vérifier si le nombre d'enregistrements récupérés est inférieur à la limite
    if len(results) < limit:
        print(f"Nombre d'enregistrements récupérés : {len(results)} à l'offset {offset}")
        print("Fin de la pagination.")
        break  # Arrêter la boucle si tous les enregistrements ont été récupérés

    # Mettre à jour le décalage pour l'itération suivante
    offset += limit

print(f"Total des enregistrements récupérés: {len(df_entreprise)}")
print(df_entreprise.head())  # Afficher les premières lignes du DataFrame


Appel de l'API avec l'URL: https://data.paysdelaloire.fr/api/explore/v2.1/catalog/datasets/120027016_base-sirene-v3-ss/records?limit=100&offset=0
Appel de l'API avec l'URL: https://data.paysdelaloire.fr/api/explore/v2.1/catalog/datasets/120027016_base-sirene-v3-ss/records?limit=100&offset=100
Appel de l'API avec l'URL: https://data.paysdelaloire.fr/api/explore/v2.1/catalog/datasets/120027016_base-sirene-v3-ss/records?limit=100&offset=200
Appel de l'API avec l'URL: https://data.paysdelaloire.fr/api/explore/v2.1/catalog/datasets/120027016_base-sirene-v3-ss/records?limit=100&offset=300
Appel de l'API avec l'URL: https://data.paysdelaloire.fr/api/explore/v2.1/catalog/datasets/120027016_base-sirene-v3-ss/records?limit=100&offset=400
Appel de l'API avec l'URL: https://data.paysdelaloire.fr/api/explore/v2.1/catalog/datasets/120027016_base-sirene-v3-ss/records?limit=100&offset=500
Appel de l'API avec l'URL: https://data.paysdelaloire.fr/api/explore/v2.1/catalog/datasets/120027016_base-sirene-v

In [24]:
df_entreprise.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4505 entries, 0 to 4504
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   siret                           4505 non-null   object 
 1   categorieentreprise             2484 non-null   object 
 2   sectionetablissement            4505 non-null   object 
 3   longitude                       4470 non-null   float64
 4   latitude                        4470 non-null   float64
 5   etatadministratifetablissement  4505 non-null   object 
dtypes: float64(2), object(4)
memory usage: 211.3+ KB


In [25]:
df_entreprise.info

<bound method DataFrame.info of                siret categorieentreprise  \
0     88353994200010                 PME   
1     44853167300011                None   
2     91890212300028                None   
3     98079521500014                None   
4     91827691600015                None   
...              ...                 ...   
4500  90251736600016                None   
4501  45316342000012                None   
4502  81260205000013                 PME   
4503  82404704700017                 PME   
4504  39416923900016                 PME   

                                   sectionetablissement  longitude   latitude  \
0                                          Construction   0.432794  48.052313   
1                                Activités immobilières  -0.442303  47.302825   
2     Activités spécialisées, scientifiques et techn...  -1.611516  47.091854   
3     Activités de services administratifs et de sou...  -1.286500  46.311899   
4     Activités spécialisées, scie

In [26]:
df_entreprise.isnull().sum()

siret                                0
categorieentreprise               2021
sectionetablissement                 0
longitude                           35
latitude                            35
etatadministratifetablissement       0
dtype: int64

In [27]:
for col in df_entreprise.select_dtypes('float64'):
    print(f'{col :-<20} {df_entreprise[col].unique()}')

longitude----------- [ 0.432794 -0.442303 -1.611516 ... -0.553487 -1.58225  -0.776818]
latitude------------ [48.052313 47.302825 47.091854 ... 47.462005 47.290779 47.248403]


In [29]:
for col in df_entreprise.select_dtypes('object'):
    print(f'{col :-<20} {df_entreprise[col].unique()}')

siret--------------- ['88353994200010' '44853167300011' '91890212300028' ... '81260205000013'
 '82404704700017' '39416923900016']
categorieentreprise- ['PME' None 'GE' 'ETI']
sectionetablissement ['Construction' 'Activités immobilières'
 'Activités spécialisées, scientifiques et techniques'
 'Activités de services administratifs et de soutien'
 "Commerce ; réparation d'automobiles et de motocycles"
 'Transports et entreposage'
 "Production et distribution d'électricité, de gaz, de vapeur et d'air conditionné"
 'Industrie manufacturière' 'Hébergement et restauration'
 'Autres activités de services' 'Agriculture, sylviculture et pêche'
 'Santé humaine et action sociale' "Activités financières et d'assurance"
 'Enseignement' 'Arts, spectacles et activités récréatives'
 'Immobilier, location et services aux entreprises'
 'Information et communication' 'Administration publique'
 'Industries extractives'
 "Production et distribution d'eau ; assainissement, gestion des déchets et dépollution"